# Packages

In [ ]:
using LinearAlgebra
using DifferentialEquations, DiffEqCallbacks
using COVID19SchoolReopening
using ProgressMeter
using PyPlot, LaTeXStrings

const CSR = COVID19SchoolReopening

rcParams = PyPlot.PyDict(PyPlot.matplotlib."rcParams")
rcParams["axes.grid"] = true
rcParams["font.size"] = 12

# Model Parameters

In [ ]:
# length of simulation, translated from months to days (model time scale)
months = 6
days = months * CSR.DAYS_PER_MONTH
weeks = Int(round(Int, days / 7))
tspan = (0, days*1.0);

In [ ]:
# assuming model time in days

# cohort interaction, between 0 and 1
α = 0.05 * ones(3, 3)
α[1,1] = 1      # cohort 1
α[2,2] = 1      # cohort 2
α[3,3] = 1      # cohort 3

# latent period, time between exposure time and beginning of infectious period
σ = zeros(2)
σ[1] = 1/(3)     # children, 3 days
σ[2] = 1/(3)     # adults, 3 days

# removal rate, 'recovery' from infectious state to non-infectious
γ = zeros(2)
γ[1] = 1/(4)    # child recovery rate, 4 days
γ[2] = 1/(4)    # adult recovery rate, 4 days

# baseline transmission rates
β0 = 1.2
F = [
    0.1     0.25    # child to j
    0.15    0.5     # adult to j
]
β = β0 * F

# population: [children, adults]
demographics = [22., 78.]
total_infected = 2.0
infected = [10., 90.]

params = (α, σ, γ);

In [ ]:
function get_compartment(u, i, j)
    c = zero(eltype(u))
    for k in 1:size(u, 3)
        c += max(0, u[i,j,k])
    end
    return c
end

infected_child_fraction(u) = get_compartment(u, 3, 1)
removed_child_fraction(u) = get_compartment(u, 4, 1)
cumulative_child_fraction(u) = infected_child_fraction(u) + removed_child_fraction(u)

infected_adult_fraction(u) = get_compartment(u, 3, 2)
removed_adult_fraction(u) = get_compartment(u, 4, 2)
cumulative_adult_fraction(u) = infected_adult_fraction(u) + removed_adult_fraction(u)

scale_percent(t, x) = (t, scale_percent(x))
scale_percent(x) = x * 100

# recording data
function record_figure_data!(data, sol, detected, threshold, window, test_times, demographics)
    # shorthand
    q = demographics / sum(demographics)
    ts = sol.t
    u = sol.u

    # find threshold time
    x = [ sum(detected[max(1,i-window):i]) for i in eachindex(detected) ]
    index = findfirst(≥(q[1]*threshold), x)
    t_thresh = isnothing(index) ? Inf : test_times[index]

    # time points in solution
    push!(data.ts, sol.t)

    # child data
    push!(data.child_school_I,  scale_percent.(sol[3,1,1,:]) ./ q[1])
    push!(data.child_remote_I,  scale_percent.(sol[3,1,2,:]) ./ q[1])
    push!(data.child_school_S,  scale_percent.(sol[1,1,1,:]) ./ q[1])
    push!(data.child_remote_S,  scale_percent.(sol[1,1,2,:]) ./ q[1])
    push!(data.child_detected,  scale_percent.(x) ./ q[1])

    push!(data.adult_school_I,  scale_percent.(sol[3,2,1,:]) ./ q[2])
    push!(data.adult_remote_I,  scale_percent.(sol[3,2,2,:]) ./ q[2])
    push!(data.adult_school_S,  scale_percent.(sol[1,2,1,:]) ./ q[2])
    push!(data.adult_remote_S,  scale_percent.(sol[1,2,2,:]) ./ q[2])

    push!(data.t_thresh,        t_thresh)

    return nothing
end

# Fig 5: Alternative cohort structure

Two cohorts: one attends school, the other opts for remote learning.
The in-person cohort has an elevated child-to-child transmission rate.

In [ ]:
# policy parameters
threshold = 0.05
sensitivity_tested = [1.0, 0.5]
window = 14

# simulation parameters
nclasses = 2
ncohorts = 2

multiplier = 10.0

# testing times - school days only; applies to active cohort
school_days = 1:1.0:5
test_times = [(day-1) + 7*(week-1) for week in 1:weeks for day in school_days];

In [ ]:
# allocate outputs
data = (
    ts                = Vector{Float64}[], # time axis
    child_school_I    = Vector{Float64}[], # I_11
    child_remote_I    = Vector{Float64}[], # I_12
    child_school_S    = Vector{Float64}[], # S_11
    child_remote_S    = Vector{Float64}[], # S_12
    child_detected    = Vector{Float64}[], # 14-window criterion
    adult_school_I    = Vector{Float64}[], # I_21
    adult_remote_I    = Vector{Float64}[], # I_22
    adult_school_S    = Vector{Float64}[], # S_21
    adult_remote_S    = Vector{Float64}[], # S_22
    t_thresh          = Float64[],         # threshold time
)

# closure to make things easier
record_figure_data!(data, sol, detected) = record_figure_data!(data, sol, detected, threshold, window, test_times, demographics)

# no intervention
prob = make_model_instance(nclasses, ncohorts, tspan, β, params,
    mult=multiplier,
    demographics=demographics,
    total_infected=total_infected,
    infected=infected,
)

# only the first cohort attends school in person
cohort_β = prob.p[4]

# cohort 1 --- in-person
ratef = cohort_β[1][1,1]
cohort_β[1][1,1] = TransmissionRate(ratef.cmax, ratef.cmin, ratef.start, ratef.stop, K=ratef.K)

# cohort 2 --- remote learning
ratef = cohort_β[2][1,1]
cohort_β[2][1,1] = TransmissionRate(ratef.cmin, ratef.cmin, ratef.start, ratef.stop, K=ratef.K)

# run simulation
detected_cases = Float64[]
sol = solve(prob)

# record data
record_figure_data!(data, sol, detected_cases)

# with intervention, perfect detection
detected_cases = Float64[]
cb_test, cb_threshold, cbs = make_threshold_policy_callbacks(test_times, detected_cases,
    sensitivity=sensitivity_tested[1],
    isrotating=false,
    threshold=threshold,
    window=window,
)
prob = make_model_instance(nclasses, ncohorts, tspan, β, params,
    mult=multiplier,
    demographics=demographics,
    total_infected=total_infected,
    infected=infected,
)

# only the first cohort attends school in person
cohort_β = prob.p[4]

# cohort 1 --- in-person
ratef = cohort_β[1][1,1]
cohort_β[1][1,1] = TransmissionRate(ratef.cmax, ratef.cmin, ratef.start, ratef.stop, K=ratef.K)

# cohort 2 --- remote learning
ratef = cohort_β[2][1,1]
cohort_β[2][1,1] = TransmissionRate(ratef.cmin, ratef.cmin, ratef.start, ratef.stop, K=ratef.K)

# run simulation
sol = solve(prob, callback=cbs, userdata=Dict(:school_closed=>false))

# record data
record_figure_data!(data, sol, detected_cases)

# with intervention, 50% detection
detected_cases = Float64[]
cb_test, cb_threshold, cbs = make_threshold_policy_callbacks(test_times, detected_cases,
    sensitivity=sensitivity_tested[2],
    isrotating=false,
    threshold=threshold,
    window=window,
)

prob = make_model_instance(nclasses, ncohorts, tspan, β, params,
    mult=multiplier,
    demographics=demographics,
    total_infected=total_infected,
    infected=infected,
)

# only the first cohort attends school in person
cohort_β = prob.p[4]

# cohort 1 --- in-person
ratef = cohort_β[1][1,1]
cohort_β[1][1,1] = TransmissionRate(ratef.cmax, ratef.cmin, ratef.start, ratef.stop, K=ratef.K)

# cohort 2 --- remote learning
ratef = cohort_β[2][1,1]
cohort_β[2][1,1] = TransmissionRate(ratef.cmin, ratef.cmin, ratef.start, ratef.stop, K=ratef.K)

# run simulation
sol = solve(prob, callback=cbs, userdata=Dict(:school_closed=>false))

# record data
record_figure_data!(data, sol, detected_cases)

In [ ]:
# background
rcParams["axes.grid"] = false
rcParams["axes.facecolor"] = "0.90"

fig, ax = subplots(figsize=[14.0, 6.0], nrows=2, ncols=3)

fig.add_subplot(frameon=false)
plt.tick_params(labelcolor="none", top=false, bottom=false, left=false, right=false)
plt.xlabel("Weeks after reopening")
plt.ylabel("Proportion (%)", labelpad=20)

common_xticks = 0:14:tspan[end] # show every two weeks
common_xtickl = string.(Int.(common_xticks .÷ 7))

ymin = 0.0
ymax = 8.0

#
#   i = 1: no intervention
#   i = 2: 100% sensitivity
#   i = 3: 50% sensitivity
#
for i in eachindex(data.ts)
    # 1,1 -- infections in children in school cohort
    x = data.child_detected[i]
    ax[1].plot(data.ts[i], data.child_school_I[i], "-", color="C$(i-1)", lw=3)
    ax[1].plot(test_times[1:length(x)], x, "--", color="C$(i-1)", lw=3)
    (i > 1) && ax[1].axvline(data.t_thresh[i], linestyle=":", color="C$(i-1)")

    # 2,1 -- infections in adults in school cohort
    ax[2].plot(data.ts[i], data.adult_school_I[i], "-", color="C$(i-1)", lw=3)
    (i > 1) && ax[2].axvline(data.t_thresh[i], linestyle=":", color="C$(i-1)")

    # 1,2 -- infections in children in home cohort
    ax[3].plot(data.ts[i], data.child_remote_I[i], "-", color="C$(i-1)", lw=3)
    (i > 1) && ax[3].axvline(data.t_thresh[i], linestyle=":", color="C$(i-1)")

    # 2,2 -- infections in adults in home cohort
    ax[4].plot(data.ts[i], data.adult_remote_I[i], "-", color="C$(i-1)", lw=3)
    (i > 1) && ax[4].axvline(data.t_thresh[i], linestyle=":", color="C$(i-1)")

    # 1,3 -- susceptible children
    y = data.child_school_S[i] + data.child_remote_S[i]
    ax[5].plot(data.ts[i], y, "-", color="C$(i-1)", lw=3)
    (i > 1) && ax[5].axvline(data.t_thresh[i], linestyle=":", color="C$(i-1)")

    # 2,3 -- susceptible adults
    y = data.adult_school_S[i] + data.adult_remote_S[i]
    ax[6].plot(data.ts[i], y, "-", color="C$(i-1)", lw=3)
    (i > 1) && ax[6].axvline(data.t_thresh[i], linestyle=":", color="C$(i-1)")
end

for i in eachindex(ax)
    ax[i].xaxis.set_ticks(common_xticks)
    ax[i].xaxis.set_ticklabels(common_xtickl)

    if i ≥ 5
        ax[i].set_ylim(75, 100)
    else
        ax[i].set_ylim(0, 1.5)
    end
end

# set labels for rows
ax[1].set_ylabel("Children")
ax[2].set_ylabel("Adults")

# set labels for columns
ax[1].set_title("School cohort")
ax[3].set_title("Remote cohort")
ax[5].set_title("Susceptibles (combined)")

# add figure legend
artist_c0 = plt.Line2D((0,1), (0,0), color="C0")
artist_c1 = plt.Line2D((0,1), (0,0), color="C1")
artist_c2 = plt.Line2D((0,1), (0,0), color="C2")

artist_solid = plt.Line2D((0,1), (0,0), color="k", linestyle="-")
artist_dash = plt.Line2D((0,1), (0,0), color="k", linestyle="--")

fig.legend(
    (artist_c0, artist_c1, artist_c2, artist_solid, artist_dash),
    ("no intervention", "100% detection", "50% detection", latexstring(L"I(t)", " / ", L"S(t)"), "detected"),
    bbox_to_anchor=(0.21,0.0,0.5,0),
    loc="lower left",
    borderaxespad=0.0,
    frameon=false,
    ncol=5)
fig.subplots_adjust(bottom=0.09)

# add labels to each subplot
reindex = [1, 4, 2, 5, 3, 6]
for i in eachindex(ax)
    ax[i].text(-0.04, 1.15, string('A'+(reindex[i]-1)), transform=ax[i].transAxes,
      fontsize=16, fontweight="bold", va="top", ha="right")
    ax[i].grid(axis="y")
end

fig.tight_layout()

PyPlot.savefig("../figures/Fig5.png", dpi=300, bbox_inches="tight")
PyPlot.savefig("../figures/Fig5.tiff", dpi=300, bbox_inches="tight")

fig

# Appendix

In [ ]:
PyPlot.close_figs()

In [ ]:
using InteractiveUtils; versioninfo()

In [ ]:
using Pkg; Pkg.status()